In [1]:
from matplotlib.colors import ListedColormap # for grgphing decision boundaries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split

In [2]:
data_folder = 'data'
X_train = pd.read_csv(f'./{data_folder}/X_train.csv')
y_train = pd.read_csv(f'./{data_folder}/y_train.csv')
X_test = pd.read_csv(f'./{data_folder}/X_test.csv')
y_test = pd.read_csv(f'./{data_folder}/y_test.csv')

In [3]:
class Network:
    def __init__(self, layers):
        self.layers, self.weights, self.biases = self.create_weights_biases_layers(layers)
        
    @classmethod
    def create_weights_biases_layers(self, layers):
        weights = []
        biases = []
        layers_vectors = [0] * len(layers)
        for i in range(len(layers)-1):
            weights.append(np.random.randn(layers[i+1], layers[i]))
            biases.append(np.random.randn(layers[i+1], 1))
            
            current_layer = [[0] for j in  range(layers[i])]
            layers_vectors[i+1] = np.reshape(current_layer, (-1,1))
        
        return layers_vectors, weights, biases
            
    def sigmoid(self, activation):
        return (1/(1+np.exp(-(activation))))

    def feed_forward(self, X):
        # select the layers
        layers = self.layers
        # set the input as the input layer
        layers[0] = np.reshape(X, (-1,1))
        
        # input to h1
        activations_input_h2 = self.sigmoid(np.dot(self.weights[0], layers[0]) + self.biases[0])
        layers[1] = activations_input_h2.copy()
        
        # h1 to h2 
        activations_h1_h2 = self.sigmoid(np.dot(self.weights[1], layers[1]) + self.biases[1])
        layers[2] = activations_h1_h2.copy()
        
        # h2 to output
        activations_h2_output = self.sigmoid(np.dot(self.weights[2], layers[2]) + self.biases[2])
        layers[3] = activations_h2_output
        
        return (layers[-1], max(layers[-1]))
    
    def back_prop(self, output, y, learning_rate=0.01):
        weights = self.weights
        biases = self.biases
        layers = self.layers
        
        new_weights = weights.copy()
        new_biases = biases.copy()
        cost_function_derivative = 2 * (output - y)
        
        # output to h2 
        
        h3_z
        dc_dw_output_h2 = cost_function_derivative.T * (self.sigmoid(layers[3])*self.sigmoid(layers[3])) * layers[2]
        new_weights[2] += (-learning_rate * dc_dw_output_h2)
        
        # h2 to h1
        dc_dw_h2_h1 = cost_function_derivative.T * (self.sigmoid(layers[2])*self.sigmoid(layers[2])) * layers[1]
        new_weights[1] += (-learning_rate * dc_dw_h2_h1)
        return 0
        
    def train(self, X, y):
        y_dummies = pd.get_dummies(y)
        data = pd.concat([X, y_dummies], axis=1)
        activations = self.feed_forward()
        return 0

In [4]:
list(X_train.iloc[0])

[5.5, 2.4, 3.7, 1.0]

In [5]:
# number of neurons at each layers input, h1, h2, output
layers = (len(X_train.iloc[0]), 4, 3, 3)

In [6]:
new_class = Network(layers=layers)

In [7]:
outputs, output = new_class.feed_forward(np.reshape(list(X_train.iloc[0]), [-1,1]))

In [8]:
# new_class.train(X_train, y_train)

In [9]:
# new_class.predict(list(X_test.iloc[0]), np.reshape(list(pd.get_dummies(y_test).iloc[0]), [-1,1]))

In [10]:
# new_class.get_accuracy(X_train, y_train)

In [11]:
# new_class.get_accuracy(X_test, y_test)


In [12]:
new_class.back_prop(outputs, np.reshape(list(pd.get_dummies(y_test).iloc[0]), [-1,1]))

NameError: name 'h3_z' is not defined